In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }/*This file contains any manual css for this page that needs to override the global styles. This is only required when different pages style the same element differently. This is just a hack to deal with our current css styles and no new styling should be added in this file.*/  #ipython-main-app {     position: relative; } #jupyter-main-app {     position: relative; }   /*Hemendik aurrera nik idatzi det*/ body {    font-family: Ubuntu, sans-serif; }  .jupyter-widgets { 	margin: 5px !important; 	font-family: Ubuntu, sans-serif !important; }  .jupyter-button { 	top: 4px !important; 	line-height: 28px !important; }  .widget-text { 	font-size: 15px !important; }  .output_subarea { 	margin: auto; }  /*formularioen stylea*/ /* naranja: #fcba05 */ /* gris: #afafaf */  input[type=text] { 	padding: 3px !important; 	border: 0 0 0 0; 	border-bottom-color: #fcba05 !important; 	border-bottom-width: 2px !important; 	 	border-top-width: 1px !important; 	border-left-width: 1px !important; 	border-right-width: 1px !important; 	 	border-top-color: transparent !important; 	border-left-color: transparent !important; 	border-right-color: transparent !important; }  input[type=text]:hover{ 	     border-top-color: #afafaf !important; 	border-left-color: #afafaf !important; 	border-right-color: #afafaf !important; }  input[type=text]:focus{     border-top-color: #fcba05 !important; 	border-left-color: #fcba05 !important; 	border-right-color: #fcba05 !important; }</style>"))

In [2]:
display(HTML("<style>.jupyter-button{margin-top:5px} .widget-hbox{min-height:50px} .widget-checkbox .widget-label{width:0px} .widget-checkbox .widget-dropdown{width:0px}"))

In [3]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa
from bqplot import pyplot as pyplot

import numpy as np

import pandas as pd

from IPython.display import display, FileLink, FileLinks
import ipywidgets as widgets

import qgrid

from traitlets import link

import sys,os
import csv

import fileupload
import io

import math

In [4]:
class dataDB(object):
    
    def __init__(self):
        self.all = {}
        self.index = 0
        
    def add(self, df, name=''):
        
        if name == '':
            name = "Data " + str(self.get_last_name())

        elif name in self.all.keys():
            name = name + " " + str(self.get_last_name())
        
        self.increment_index()

        #df = check_df_header(df)
        self.all[name] = df
        return name
    
    def update(self, df, name):
        self.all[name] = df
        
    def get(self, name):
        if name not in self.all.keys():
            return pd.DataFrame({"x": float(np.linspace(0.00, 0.00,1)), "y": np.linspace(0.00, 0.00,1)})
        else:        
            return self.all.get(name)
    
    def remove(self, name):
        if len(self.all) > 1:
            del self.all[name]
        
    def get_last_name(self):
        return self.index
    
    def increment_index(self):
        self.index = self.index + 1
        
    def show(self):
        return self.all
    
    def get_datas(self):
        return self.all.keys()

In [5]:
data_database = dataDB()

In [6]:
class Mark(object):
    
    def __init__(self, reg, order, dec):
        
        self.reg = reg
        self.order = order
        self.dec = dec
        
    def update(self, reg, order, dec):
        
        self.reg = reg
        self.order = order
        self.dec = dec

In [7]:
class Plot(object):
    
    def __init__(self, name, df, figure, col_x, col_y):
        
        self.name = name
        self.df = df
        self.figure = bq.Figure(
            title = figure.title,
            axes = figure.axes,
            marks = figure.marks,
            scale_x = figure.scale_x,
            scale_y = figure.scale_y,
            legend_location = figure.legend_location,
            animation_duration = figure.animation_duration,            
        )
        self.columns = [(col_x, col_y)]
        self.marks = [Mark(False, 1, 2)]
        
    def update(self, name, df, figure, col_x, col_y, marks):
        
        self.name = name
        self.df = df
        self.figure = bq.Figure(
            title = figure.title,
            axes = figure.axes,
            marks = figure.marks,
            scale_x = figure.scale_x,
            scale_y = figure.scale_y,
            legend_location = figure.legend_location,
            animation_duration = figure.animation_duration,            
        )
        self.columns.append((col_x, col_y))
        
        self.marks = marks

In [8]:
class plotDB(object):
    
    def __init__(self):
        self.all = {}
        self.plots = {}
        self.plots_index = 0
        
    def add_plot(self, df, figure, col_x, col_y, name=''):
        if name == '':
            name = "Plot " + str(self.plots_index)
        elif name in self.plots.keys():
            name = name + " " + str(self.plots_index)
        p = Plot(name, df, figure, col_x, col_y)
        self.all[name] = p
        self.plots[name] = p
        
        self.plots_index = self.plots_index + 1
    
    def update_plot(self, df, figure, col_x, col_y, marks, name):
        p = self.get_plot(name)
        p.update(name, df, figure, col_x, col_y, marks)
        self.all[name] = p
        self.plots[name] = p
        
    def save_plot(self, df, figure, name, marks):
        p = self.get_plot(name)
        p.df = df
        p.figure = bq.Figure(
            title = figure.title,
            axes = figure.axes,
            marks = figure.marks,
            scale_x = figure.scale_x,
            scale_y = figure.scale_y,
            legend_location = figure.legend_location,
            animation_duration = figure.animation_duration,            
        )
        p.marks = marks
        
        self.all[name] = p
        self.plots[name] = p
    
    def get_plot(self, name):
        return self.plots.get(name)
    
    def remove_plot(self, name):
        if len(self.plots) > 1:
            del self.all[name]
            del self.plots[name]
            
    def get_plots(self):
        return self.plots.keys()
    
    def get_index(self):
        return self.plots_index

In [9]:
plot_database = plotDB()

In [10]:
separators = {
    ',':',',
    ';':';',
    'tab': '\t',
    'whitespace': ' ',
}

In [11]:
def change_import_menu(button):
    """This function is called when a button from
    import_buttons_block is clicked and changes the menu showed in 
    import_menu_block box.
    
    """
    
    if button.description == "OPEN from PC":
        import_menu_block.children = [import_pc_block,]
        
    elif button.description == "OPEN from SERVER":
        import_menu_block.children = [import_server_block,]
    
    elif button.description == "NEW blank CHART":
        import_menu_block.children = [blank_chart_block,]

In [12]:
def open_csv(file, separator=','):
    """This function tries to open the csv file 
    with a given separator and returns a pandas dataframe.
    
    Returns:
        A pandas dataframe if the file could be opened or
        None if there was any error.
    
    """
    
    try:
        df = pd.read_csv(
            file,
            sep=separator,
            dtype=np.float64
        )
        
        return df
    
    except:
        
        return None

In [13]:
def import_file_from_pc(change):
    """This function is called by a FileUpload IPython widget
    when a file is uploaded.
    It calls to open_csv function with the file provided in
    change.owner.data and the separator from
    import_pc_separator.value
    
    """
    
    # IMPORTANT:
    # THIS FUNCTION HAS BEEN MODIFIED TO WORK WITH FILEUPLOAD
    # INSTEAD OF THE FILEUPLOAD IPYTHON WIDGET.
    # IN ORDER TO REVERT THE CHANGES THE LINE 18 AND 30 MUST
    # BE REPLACED BY THE COMMENTED ONES ABOVE THEM
    
    button = change.owner

    #datastr = str(button.data[0],'utf-8')
    datastr = str(button.data,'utf-8')
    
    datafile = io.StringIO(datastr)
    
    open_message.value = "<b>Uploading file...</b>"
    df = open_csv(
        datafile,
        separators[import_pc_separator.value],
    )
    
    if df is not None:
        #filename = list(import_pc_file.value.keys())[0]
        filename = button.filename
        
        open_message.value = "<span style='color:green'><b>'" + filename + "' file successfully loaded</b></span>"
        
        add_new_data(df)
        
        main_chart.df = df
        
        top_block.children = [
            open_block,
            chart_block
        ]
        
    else:
        open_message.value = "<span style='color:red'><b>An error has occurred, choose another file</b></span>"

In [14]:
def import_file_from_server(a):
    """This function is called by a button IPython widget.
    
    """
    
    datafile = import_server_path.value
    
    open_message.value = "<b>Opening file...</b>"
    df = open_csv(
        datafile,
        separators[import_server_separator.value]
    )
    
    if df is not None:
        filename = datafile
        open_message.value = "<span style='color:green'><b>'" + filename + "' file successfully loaded</b></span>"
        
        add_new_data(df)
        
        main_chart.df = df
        
        top_block.children = [
            open_block,
            chart_block
        ]
        
    else:
        open_message.value = "<span style='color:red'><b>An error has occurred, choose another file</b></span>"

In [15]:
def new_blank_chart(a):
    """This function creates an empty dataframe with two columns.
    
    """
    
    df = pd.DataFrame([np.array([0.0, 0.0])], columns=['A', 'B'])
    
    open_message.value = "<span style='color:green'>Chart successfully loaded</b></span>"
    
    add_new_data(df)
    
    main_chart.df = df
    
    top_block.children = [
            open_block,
            chart_block
        ]

In [16]:
def add_new_data(df):
    """This function adds a data to the database, replaces the
    main_chart dataframe and refreshes the choose_data_dropdown.
    """
    
    data_name = data_database.add(df)
    
    choose_data_dropdown.options = data_database.get_datas()
    choose_data_dropdown.value = data_name

In [17]:
open_block = widgets.VBox(
    [],
    layout = widgets.Layout(
        border='solid 1px',
        padding='20px',
        align_items = 'center'
    )
)

import_buttons_block = widgets.HBox(
    [],
    layout = widgets.Layout(
        align_items = 'center'
    ) 
)

import_menu_block = widgets.HBox(
    []
)

open_message = widgets.HTML(
    value = "",
) 

open_block.children = [
    import_buttons_block,
    import_menu_block,
    open_message
]

import_buttons_text = [
    "OPEN from PC",
    "OPEN from SERVER",
    "NEW blank CHART"
]

import_button_block_children = []

for text in import_buttons_text:
    
    button = widgets.Button(
            description = text,
            disabled=False,
            tooltip='Click me',
    )
    
    button.on_click(change_import_menu)
    
    import_button_block_children.append(button)

import_buttons_block.children = import_button_block_children

In [18]:
def get_csv_files(files_dict):
    """This function find the .csv files avalaiables in the
    folder where this file is located.
    
    Returns:
        returns the dict files with the added csv files
        dict[file_relative_path] = file_absolute_path
        
    """
    
    root = os.getcwd()

    for path, subdirs, files in os.walk(root):
        for name in files:
            if name.endswith('.csv'):
                p = os.path.join(path, name)
                f = p.replace(root + os.sep, '')
                files_dict[f] = p

    return files_dict

In [19]:
def update_file_path(change):
    import_server_path.value = server_files.get(import_server_files.value)

In [20]:
def upload_file_button():
    """Create an upload files button
    that prints the file name and file size.
    
    """

    _upload_widget = fileupload.FileUploadWidget()

    def _cb(change):
        decoded = io.StringIO(change['owner'].data.decode('utf-8'))
        filename = change['owner'].filename

    _upload_widget.observe(_cb, names='data')
    return _upload_widget

In [21]:
import_pc_block = widgets.HBox(
    [],
    layout = widgets.Layout(
        align_items = 'center',
        width = '100%'
    ) 
)

import_pc_name = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)

import_pc_separator = widgets.Dropdown(
    options= list(separators),
    value=list(separators)[0],
    description="<h4>Separator: </h4>",
    disabled=False,
)

import_pc_file = widgets.FileUpload(
    accept='',
    multiple=False
)

import_pc_file.observe(import_file_from_pc, '_counter')

import_pc_file_button = upload_file_button()
import_pc_file_button.observe(import_file_from_pc, 'data')

import_pc_block.children = [
    import_pc_name,
    import_pc_separator,
    #import_pc_file,
    import_pc_file_button
]

In [22]:
import_server_block = widgets.VBox(
    [],
    layout = widgets.Layout(
        align_items = 'center',
        width = '100%'
    ) 
)

import_server_name = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)

import_server_separator = widgets.Dropdown(
    options= list(separators),
    value=list(separators)[0],
    description="<h4>Separator: </h4>",
    disabled=False,
)

server_files = { '-Select file-': ''}
server_files = get_csv_files(server_files)
files_list = list(server_files.keys())


import_server_files = widgets.Dropdown(
    options= files_list,
    #value='',
    description="<h4>File: </h4>",
    disabled=False,
)

import_server_files.observe(update_file_path, 'value')

import_server_path = widgets.Text(
    value= '',
    placeholder="Type your file's path:",
    description="<h4>Path: </h4>",
    disabled=False,
    layout = {'width' : '80%'}
)

import_server_button = widgets.Button(
    description = 'Open file',
    disabled=False,
    tooltip='',
    )
    
import_server_button.on_click(import_file_from_server)

import_server_block.children = [
    widgets.HBox(
        [
            import_server_name,
            import_server_separator,
            import_server_files
        ],
        layout = widgets.Layout(
            align_items = 'center',
            width = '100%'
        ) 
    ),
    widgets.HBox(
        [
            import_server_path,
            import_server_button,
        ],
        layout = widgets.Layout(
            align_items = 'center',
            width = '100%'
        )
    )
]

In [23]:
blank_chart_block = widgets.HBox(
    [],
    layout = widgets.Layout(
        align_items = 'center',
        width = '100%'
    ) 
)

blank_chart_name = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)

blank_chart_button = widgets.Button(
    description = 'New blank chart',
    disabled=False,
    tooltip='',
    )
    
blank_chart_button.on_click(new_blank_chart)

blank_chart_block.children = [
    blank_chart_name,
    blank_chart_button
]

In [24]:
open_block.children = [
    import_buttons_block,
    import_menu_block,
    open_message
]

In [25]:
def refresh_column_dropdowns(dropdowns, df):
    """This function updates the options of all dropdown
       IPython widgets given in dropdowns with the columns of 
       df pandas dataframe.
       
    """
    
    for dropdown in dropdowns:
        dropdown.options = df.columns.values

In [26]:
def get_unique_name(new_name, name_list):
    """This function checks if new_name is in name_list
       and if it is or if new_name = '' returns a string containing
       a name that is not present in name_list based on
       alphabetical characters.
    
    """
    
    if new_name=='' or new_name in name_list:
        
        alphabet = [
            'A','B','C',
            'D','E','F',
            'G','H','I',
            'J','K','L',
            'M','N','O',
            'P','Q','R',
            'S','T','U',
            'V','W','X',
            'Y','Z'
        ]
        new_alphabet = [elem for elem in alphabet]
        
        counter = 0
        
        while True:
            for elem in new_alphabet:
                if elem not in name_list:

                    return elem
            
            first_char = alphabet[counter]
            
            new_alphabet = [first_char+char for char in alphabet]
            counter = counter + 1
            
    else:
        
        return new_name

In [27]:
def add_column(a):
    """This function adds an empty column to main_chart using the name
       given at new_col_name (if provided).
    
    """
    
    df = main_chart.get_changed_df()
    
    column_names = df.columns.values
    new_col_name = get_unique_name(
        new_column_name.value,
        column_names
    )
    
    df[new_col_name] = 0.0
    main_chart.df = df
    
    refresh_column_dropdowns(
        column_dropdowns,
        main_chart.df
    )
    
    new_column_name.value = ''

In [28]:
def rename_column(a):
    """This function renames the column selected in
       choose_column_dropdown with the name provided in 
       rename_colomn_name.
    
    """
    
    df = main_chart.get_changed_df()
    column_names = df.columns.values
    
    col = choose_column_dropdown.value
    new_name = get_unique_name(
        rename_column_name.value,
        column_names
    )
    
    df.rename(columns={col: new_name}, inplace=True)
    
    main_chart.df = df

    refresh_column_dropdowns(
        column_dropdowns,
        main_chart.df
    )
    
    rename_column_name.value = ''

In [29]:
def remove_column(a):
    """This function removes the column selected in
       choose_column_dropdown.
    
    """
    
    df = main_chart.get_changed_df()
    
    if len(df.columns.values) > 1:
    
        col = choose_column_dropdown.value

        df.drop(columns=[col], inplace=True)

        main_chart.df = df

        refresh_column_dropdowns(
            column_dropdowns,
            main_chart.df
        )

In [30]:
def change_chart_df(change):
    
    obj = change.owner
    new_df_name = obj.value
    
    main_chart.df = data_database.get(new_df_name)
    
    refresh_column_dropdowns(
            column_dropdowns,
            main_chart.df
        )

In [31]:
def export_main_chart(a):
    
    filename = export_main_chart_name.value
    
    if filename == '':
        filename = choose_data_dropdown.value.replace(' ', '_')
        
    df = main_chart.get_changed_df()
    fmt = choose_main_chart_format.value
    
    export_chart(
        df,
        fmt,
        filename,
        export_main_chart_text
    )
    
    export_main_chart_name.value = ''

In [32]:
chart_block = widgets.VBox(
    [],
    layout = widgets.Layout(
        border='solid 1px',
        padding='20px',
        align_items = 'center',
        width='100%'
    )
)

new_column_name = widgets.Text(
    value='',
    placeholder="New column's name:",
    description='',
    disabled=False
)

add_column_button = widgets.Button(
    description = 'Add column',
)

add_column_button.on_click(add_column)

choose_column_dropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose a column: </h4>',
    style = {
        'description_width' : 'initial'
    }
)

rename_column_name = widgets.Text(
    value='',
    placeholder="Column's new name:",
    description='',
    disabled=False
)

rename_column_button = widgets.Button(
    description = 'Rename column',
)

rename_column_button.on_click(rename_column)

remove_column_button = widgets.Button(
    description = 'Remove column',
)

remove_column_button.on_click(remove_column)

#add_row_button = widgets.Button(
#    description = 'Add row',
#)
#
#remove_row_button = widgets.Button(
#    description = 'Remove row',
#)

choose_data_dropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose a data: </h4>',
    style = {
        'description_width' : 'initial'
    }
)

choose_data_dropdown.observe(change_chart_df, 'value')

main_chart = qgrid.QGridWidget(
    df = pd.DataFrame([np.array([0.0, 0.0])], columns=['A', 'B']),
    precision = 6,
    editable = True,
    show_toolbar = True
)

export_main_chart_name = widgets.Text(
    value='',
    placeholder="File's name:",
    description='',
    disabled=False
)

choose_main_chart_format = widgets.Dropdown(
    options = ['CSV', 'TEX'],
    #description='<h4>Choose a plot: </h4>',
    style = {
        'description_width' : 'initial'
    }
)

export_main_chart_button = widgets.Button(
    description = 'Export chart',
)

export_main_chart_button.on_click(export_main_chart)

export_main_chart_text = widgets.HTML()

chart_block.children = [
    widgets.HBox(
        [
            new_column_name,
            add_column_button
        ],
        layout = widgets.Layout(
            align_items = 'center',
        )
    ),
    widgets.HBox(
        [
            choose_column_dropdown,
            rename_column_name,
            rename_column_button,
            remove_column_button
        ],
        layout = widgets.Layout(
            align_items = 'center',
        )
    ),
    #widgets.HBox(
    #    [
    #        add_row_button,
    #        remove_row_button,
    #    ],
    #    layout = widgets.Layout(
    #        align_items = 'center',
    #    )
    #),
    widgets.HBox(
        [
            choose_data_dropdown,
        ],
        layout = widgets.Layout(
            align_items = 'center',
        )
    ),
    widgets.HBox(
        [
            main_chart,
        ],
        layout = widgets.Layout(
            width = '100%',
        )
    ),
    widgets.HBox(
        [
            export_main_chart_name,
            choose_main_chart_format,
            export_main_chart_button
        ],
        layout = widgets.Layout(
            width = '100%',
            align_items = 'center',
        )
    ),
    export_main_chart_text
]

In [33]:
def is_not_number(s):
    """This function checks if the string s
    can be interpreted as a number.
    
    """
    try:
        float(s)
    except:
        return True
    else:
        return False

In [34]:
def calculate_column(a):
    """This function calculates the value of a 
    column based in an arithmetical operation between
    the columns of a dataframe.
    
    """
    op = op_col_dropdown.value
    c1 = const1.value
    c2 = const2.value
    c3 = const3.value
    c4 = const4.value

    if is_not_number(c1) or is_not_number(c2) or is_not_number(c3) or is_not_number(c4):
        return None

    else:
        c1 = float(c1)
        c2 = float(c2)
        c3 = float(c3)
        c4 = float(c4)

    df = main_chart.get_changed_df()    
        
    a = np.array(df[a_col_dropdown.value])
    b = np.array(df[b_col_dropdown.value])

    with np.errstate(divide='ignore'):
        a = a**c2
        a[a == np.inf] = 0
        a = np.nan_to_num(a)

        b = b**c4
        b[b == np.inf] = 0
        b = np.nan_to_num(b)

    if op == '+':
        new = c1*a + c3*b

    elif op == '-':
        new = c1*a - c3*b

    elif op == 'x':
        new = (c1*a) * (c3*b)


    elif op == '/':

        with np.errstate(divide='ignore', invalid='ignore'):
            c = np.true_divide(c1*a,c3*b)
            c[c == np.inf] = 0
            new = np.nan_to_num(c)

    df[final_col_dropdown.value] = new

    main_chart.df = df

In [35]:
operation_box = widgets.VBox(
    [],
    layout = widgets.Layout(
            align_items = 'center',
        )
)

operation_text = widgets.HTML(
    value = "You can operate with the colums: <b>new column = (constant x column1 ^ constant) operation (constant x column2 ^ constant)</b>"    
)

final_col_dropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='100px')
)

equals = widgets.HTMLMath(
    value = '\( = \)',
)

operation_Expression = widgets.HTMLMath(
    value = '\( C1 = a \cdot C2^{b} * c \cdot C3^{d}\)',
)

power = widgets.HTMLMath(
    value = '^',
)

const1 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

const2 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

const3 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

const4 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

dot = widgets.HTMLMath(
    value = '\( \cdot \)',
)

a_col_dropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='100px')
)

op_col_dropdown = widgets.Dropdown(
    options = ['+','-','x','/'],
    description='',
    layout = widgets.Layout(width='50px')
)

b_col_dropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='100px')
)

operate_button = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='',
    layout = widgets.Layout(
        align_self = 'center'
    )
)

operate_button.on_click(calculate_column)

operation_box.children = [
    operation_text,
    widgets.HBox([
        final_col_dropdown,
        equals,
        const1,
        dot,
        a_col_dropdown,
        power,
        const2,
        op_col_dropdown,
        const3,
        dot,
        b_col_dropdown,
        power,
        const4,
        operate_button
    ])
]

In [36]:
def replace_plot(a):
    
    df = main_chart.get_changed_df()
    
    x_col_name = x_selector.value
    y_col_name = y_selector.value
    
    x_values = df[x_col_name]
    y_values = df[y_col_name]
    
    min_x, max_x, min_y, max_y = calculate_limits(
        x_values,
        y_values,
        #log_x = log_x,
        #log_y = log_y,
    )
    
    
    lin_scale_x.min = min_x
    lin_scale_x.max = max_x
    
    #log_scale_x.min = min_x/1.2
    #log_scale_x.max = max_x*1.2
    
    lin_scale_y.min = min_y
    lin_scale_y.max = max_y
    
    #log_scale_y.min = min_y/1.2
    #log_scale_y.max = max_y*1.2
    
    axis_x_name.value = x_col_name
    axis_y_name.value = y_col_name
    
    #log_x = x_scale_dropdown.value == 'Logarithmic scale'
    #log_y = y_scale_dropdown.value == 'Logarithmic scale'
    
    #if log_x:
    #    scale_x = log_scale_x
    #    
    #else:
    #    scale_x = lin_scale_x
    #    
    #if log_y:
    #    scale_y = log_scale_y
    #    
    #else:
    #    scale_y = lin_scale_y
    #
    new_plot = bq.Figure(
        title = 'My plot',
        axes = main_plot.axes,
        marks = [
            bqm.Scatter(
                x = x_values,
                y = y_values,
                scales = {'x': lin_scale_x,
                          'y': lin_scale_y},
                colors = [points_color.value],
                labels = ['Scatter 0'],
                default_opacities = [1.0]
            ),
            bqm.Lines(
                x = [],
                y = [],
                scales = {'x': lin_scale_x,
                          'y': lin_scale_y},
                colors = [points_color.value],
                opacities = [1.0]
            )
        ],
        scale_x = lin_scale_x, #main_plot.scale_x,
        scale_y = lin_scale_y, #main_plot.scale_y,
        legend_location = main_plot.legend_location,
        animation_duration = main_plot.animation_duration,            
    )
    
    data = {
        x_col_name : x_values,
        y_col_name : y_values
    }
    
    new_df = pd.DataFrame(data = data)

    plot_database.add_plot(
        new_df,
        new_plot,
        x_col_name,
        y_col_name
    )
    
    #plot_chart.df = new_df
    
    update_plot_dropdown()
    choose_plot_dropdown.value = choose_plot_dropdown.options[-1]
    
    main_block.children = [
        top_block,
        middle_block
    ]
    
    add_button.disabled = False

In [37]:
def add_to_plot(a):
    
    df = main_chart.get_changed_df()

    x_col_name = x_selector.value
    y_col_name = y_selector.value
    
    x_values = df[x_col_name]
    y_values = df[y_col_name]
    
    name = choose_plot_dropdown.value
    p = plot_database.get_plot(name)

    log_x = x_scale_dropdown.value == 'Logarithmic scale'
    log_y = y_scale_dropdown.value == 'Logarithmic scale'
    
    min_x, max_x, min_y, max_y = calculate_limits(
        x_values,
        y_values,
        #log_x = log_x,
        #log_y = log_y,
    )
    
    lin_scale_x.min = min(min_x, lin_scale_x.min)
    lin_scale_x.max = max(max_x, lin_scale_x.max)
    
    lin_scale_y.min = min(min_y, lin_scale_y.min)
    lin_scale_y.max = max(max_y, lin_scale_y.max)
    
    log_scale_x.min = min(min_x/1.2, log_scale_x.min)
    log_scale_x.max = max(max_x*1.2, log_scale_x.max)
    
    log_scale_y.min = min(min_y/1.2, log_scale_y.min)
    log_scale_y.max = max(max_y*1.2, log_scale_y.max)
    
    if log_x:
        scale_x = log_scale_x
    
    else:
        scale_x = lin_scale_x
        
    if log_y:
        scale_y = log_scale_y
        
    else:
        scale_y = lin_scale_y
    #axis_x_name.value = x_col_name
    #axis_y_name.value = y_col_name
    
    label = 'Scatter ' + str(int(len(main_plot.marks)/2))
    
    main_plot.marks = main_plot.marks + [
        bqm.Scatter(
            x = x_values,
            y = y_values,
            scales = {'x': scale_x,
                      'y': scale_y},
            colors = [points_color.value],
            labels = [label],
            default_opacities = [1.0]
        ),
        bqm.Lines(
            x = [],
            y = [],
            scales = {'x': scale_x,
                      'y': scale_y},
            colors = [points_color.value],
            opacities = [1.0]
        )
    ]    
    
    data = {
        x_col_name : x_values,
        y_col_name : y_values
    }
      
    old_df = p.df    
    new_df = pd.DataFrame(data = data)
    
    final_data = {}
    
    for new_col in new_df.columns:
        if new_col not in old_df.columns:
            final_data[new_col] = new_df[new_col]
    
    for old_col in old_df.columns:
        final_data[old_col] = old_df[old_col]
        
    final_df = pd.DataFrame(data = final_data)
    
    #final_df = pd.concat(
    #    [new_df, old_df],
    #    axis=1,
    #    #ignore_index=True,
    #    sort=False,
    #)
    #
    #duplicated = final_df.columns.duplicated()
    #
    #for repeated in duplicated:
    #    if repeated:
    #        final_df.drop(index=[repeated])
    
    #final_df = old_df.append(
    #    new_df,
    #    #ignore_index=True,
    #    sort=False
    #)
    
    
    marks = p.marks
    marks.append(
        Mark(
        False,
        1,
        2
        )
    )
    
    plot_database.update_plot(
        final_df,
        main_plot,
        x_col_name,
        y_col_name,
        marks,
        name,
    )
    
    plot_chart.df = final_df
    
    update_mark_dropdown()

In [38]:
def update_plot_dropdown():
    
    plots = plot_database.get_plots()
    choose_plot_dropdown.options = list(plots)

In [39]:
def update_marks(name):
    
    p = plot_database.get_plot(name)
    marks = p.marks
    i = mark_dropdown.index
    
    marks[i].update(
        show_regression_checkbox.value,
        regression_order_slider.value,
        regression_decimals_dropdown.value
    )
    
    return marks

In [40]:
def change_plot(change):
    
    old = change.old

    if old is not None:
        
        marks = update_marks(old)
        
        plot_database.save_plot(
            plot_chart.get_changed_df(),
            main_plot,
            old,
            marks
        )
    
    obj = change.owner
    value = obj.value

    plot = plot_database.get_plot(value)
    df = plot.df
    figure = plot.figure

    plot_chart.df = df
    
    main_plot.title = figure.title
    main_plot.axes = figure.axes
    main_plot.marks = figure.marks
    main_plot.scale_x = figure.scale_x
    main_plot.scale_y = figure.scale_y
    main_plot.legend_location = figure.legend_location
    main_plot.animation_duration = figure.animation_duration
    
    
    if type(main_plot.scale_x) == bqs.LogScale:
        
        if type(main_plot.scale_y) == bqs.LogScale:
            x_scale_dropdown.value, y_scale_dropdown.value = 'Logarithmic scale', 'Logarithmic scale'
            
        else:
            x_scale_dropdown.value, y_scale_dropdown.value = 'Logarithmic scale', 'Linear scale'
        
    else:
        x_scale_dropdown.value, y_scale_dropdown.value = 'Linear scale', 'Linear scale'

    
    min_x, max_x, min_y, max_y = calculate_limits(
        [mark.x for mark in figure.marks[::2]], #bikoitiak bakarrik! Bakoitiak: puntuak, bikoitiak: erregresioa
        [mark.y for mark in figure.marks[::2]],
        #log_x = log_x,
        #log_y = log_y,
    )
    
    lin_scale_x.min = min_x
    lin_scale_x.max = max_x
    
    lin_scale_y.min = min_y
    lin_scale_y.max = max_y
    
    log_scale_x.min = min_x/1.2
    log_scale_x.max = max_x*1.2
    
    log_scale_y.min = min_y/1.2
    log_scale_y.max = max_y*1.2
    
    save_mark_settings(change)
    update_mark_dropdown()
    update_mark_settings(None)
    
    main_block.children = [
        top_block,
        middle_block
    ]

In [41]:
plotter_box = widgets.HBox(
    [],
    layout = widgets.Layout(
            align_items = 'center',
        )
)

y_selector = widgets.Dropdown(
    options = [],
    description='<h6>Y axis: </h6>',
    layout = widgets.Layout(width = '200px')
)

x_selector = widgets.Dropdown(
    options = [],
    description='<h6>X axis: </h6>',
    rows = 10,
    layout = widgets.Layout(width = '200px')
)

points_color = widgets.ColorPicker(
    concise=False,
    description='',
    value='#7800c8',
    disabled=False,
    layout = widgets.Layout(width = '200px')
)


replace_button = widgets.Button(
    description = 'New plot',
)

replace_button.on_click(replace_plot)

add_button = widgets.Button(
    description = 'Add to plot',
    disabled = True
)

add_button.on_click(add_to_plot)

plotter_box.children = [
    x_selector,
    y_selector,    
    points_color,
    replace_button,
    add_button
]

In [42]:
column_dropdowns = [
    final_col_dropdown,
    a_col_dropdown,
    b_col_dropdown,
    choose_column_dropdown,
    x_selector,
    y_selector
]

In [43]:
chart_block.children = chart_block.children + (operation_box, plotter_box)

In [44]:
top_block = widgets.VBox(
    [],
    layout = widgets.Layout(
            align_items = 'center',
        )
)

top_block.children = [
    open_block,
]

In [45]:
def export_to_file(df, filename, fmt):
    """This function exports a pandas dataframe
    to a selected format (.csv or .tex).
    
    """
    
    try:
        if fmt in ['CSV', 'TEX']:

            if fmt == 'CSV':

                if not filename.endswith('.csv'):
                    filename = filename + '.csv'

                file = df.to_csv(filename, index=False)

                return 'CSV', filename
            
            if fmt == 'TEX':

                if not filename.endswith('.tex'):
                    filename = filename + '.tex'

                file = df.to_latex(filename, index=False)
                
                return 'TEX', filename
            
    except:
        
        return None

In [46]:
def export_chart(df, fmt, filename, text_widget):
    
    text_widget.value = "<b>Creating file...</b>"
    
    file, filename = export_to_file(df, filename, fmt)
    
    if file is not None:
        text_widget.value = "<span style='color:green'><b>" + file +" file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download</button></form>"

        import_server_files.options = list(
            get_csv_files(server_files).keys()
        )
        
    else:
        text_widget.value = "<span style='color:red'><b>An error has occurred</b></span>"

In [47]:
def export_plot_chart(a):
    
    filename = export_plot_chart_name.value
    
    if filename == '':
        filename = choose_plot_dropdown.value.replace(' ', '_')
        
    df = plot_chart.get_changed_df()
    fmt = choose_plot_chart_format.value
    
    export_chart(
        df,
        fmt,
        filename,
        export_plot_chart_text
    )
    
    export_plot_chart_name.value = ''

In [48]:
def update_plot(event, qgrid_widget):
    
    if event.get('name') == 'cell_edited':
    
        edited_col = event.get('column')

        name = choose_plot_dropdown.value

        p = plot_database.get_plot(name)

        for i in range(len(p.columns)):

            cols = p.columns[i]

            col_x = cols[0]
            col_y = cols[1]

            new_x = plot_chart.get_changed_df()[edited_col]
            new_y = plot_chart.get_changed_df()[edited_col] 

            log_x = x_scale_dropdown.value == 'Logarithmic scale'
            log_y = y_scale_dropdown.value == 'Logarithmic scale'
            
            min_x, max_x, min_y, max_y = calculate_limits(
                new_x,
                new_y,
                #log_x = log_x,
                #log_y = log_y
            )

            if edited_col == col_x:

                main_plot.marks[2*i].x = new_x

                lin_scale_x.min = min(min_x, lin_scale_x.min)
                lin_scale_x.max = max(max_x, lin_scale_x.max)
                
                log_scale_x.min = min(min_x/1.2, log_scale_x.min)
                log_scale_x.max = min(max_x*1.2, log_scale_x.max)
    
            if edited_col == col_y:

                main_plot.marks[2*i].y = new_y

                lin_scale_y.min = min(min_y, lin_scale_y.min)
                lin_scale_y.max = max(max_y, lin_scale_y.max)
                
                log_scale_y.min = min(min_y/1.2, log_scale_y.min)
                log_scale_y.max = max(max_y*1.2, log_scale_y.max)
                
    elif event.get('name') in ('row_removed', 'filter_changed'):
        
        name = choose_plot_dropdown.value

        p = plot_database.get_plot(name)
        
        for i in range(len(p.columns)):

            cols = p.columns[i]

            col_x = cols[0]
            col_y = cols[1]

            new_x = plot_chart.get_changed_df()[col_x]
            new_y = plot_chart.get_changed_df()[col_y] 

            log_x = x_scale_dropdown.value == 'Logarithmic scale'
            log_y = y_scale_dropdown.value == 'Logarithmic scale'
            
            min_x, max_x, min_y, max_y = calculate_limits(
                new_x,
                new_y,
                #log_x = log_x,
                #log_y = log_y
            )

            main_plot.marks[i].x = new_x

            lin_scale_x.min = min(min_x, lin_scale_x.min)
            lin_scale_x.max = max(max_x, lin_scale_x.max)
            
            log_scale_y.min = min(min_y/1.2, log_scale_y.min)
            log_scale_y.max = max(max_y*1.2, log_scale_y.max)

            main_plot.marks[i].y = new_y

            lin_scale_y.min = min(min_y, lin_scale_y.min)
            lin_scale_y.max = max(max_y, lin_scale_y.max)
            
            log_scale_y.min = min(min_y/1.2, log_scale_y.min)
            log_scale_y.max = max(max_y*1.2, log_scale_y.max)
            
    update_regression(None)

In [49]:
left_block = widgets.VBox(
    [],
    layout = widgets.Layout(
        border='solid 1px',
        padding='20px',
        align_items = 'center',
        width='50%'
    )    
)

choose_plot_dropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose a plot: </h4>',
    style = {
        'description_width' : 'initial'
    }
)

choose_plot_dropdown.observe(change_plot, 'value')

plot_chart = qgrid.QGridWidget(
    df = pd.DataFrame([np.array([0.0, 0.0])], columns=['A', 'B']),
    precision = 6,
    editable = True,
    show_toolbar = True
)

instances = [
    'cell_edited',
    'row_removed',
    'filter_changed'
]

plot_chart.on(instances, update_plot)

export_plot_chart_name = widgets.Text(
    value='',
    placeholder="File's name:",
    description='',
    disabled=False
)

choose_plot_chart_format = widgets.Dropdown(
    options = ['CSV', 'TEX'],
    #description='<h4>Choose a plot: </h4>',
    style = {
        'description_width' : 'initial'
    }
)

export_plot_chart_button = widgets.Button(
    description = 'Export chart',
)

export_plot_chart_button.on_click(export_plot_chart)

export_plot_chart_text = widgets.HTML()

left_block.children = [
    choose_plot_dropdown,
    widgets.HBox(
        [
            plot_chart,
        ],
        layout = widgets.Layout(
            width = '100%',
        )
    ),
    widgets.HBox(
        [
            export_plot_chart_name,
            choose_plot_chart_format,
            export_plot_chart_button
        ],
        layout = widgets.Layout(
            width = '100%',
        )
    ),
    export_plot_chart_text
]

In [50]:
def log_round(x):
    """This function returns the nearest 10th power of a
    number."""
    
    
    if x > 1.0:
        return math.ceil(math.log10(x)) #https://www.tutorialspoint.com/python/number_ceil.htm
    
    else:
        for i in range(len(str(x))):
            digit = str(x)[i]
            if digit != '0' and digit != '.':
                return -i

In [51]:
def generate_log_ticks(log_scale):
    
    min_power = log_round(log_scale.min)
    max_power = log_round(log_scale.max)

    tick_values = [10**i for i in range(min_power, max_power)]
    ticks = []
    
    for tick in tick_values:
        if tick < log_scale_x.max and tick > log_scale_x.min:
                ticks.append(tick)
                
    return ticks

In [52]:
def change_scale(change):
    """This function changes the scale of an axis
    of main_plot.
    
    """
    obj = change.owner
    value = obj.value
    desc = obj.description

    
    if value == 'Linear scale':
        
        if 'X' in desc:
            axis_x.scale = lin_scale_x
            axis_x.tick_values = None
            axis_x.num_ticks = 5
        
        elif 'Y' in desc:
            axis_y.scale = lin_scale_y
            axis_y.tick_values = None
            axis_y.num_ticks = 4
            
    elif value == 'Logarithmic scale':

        if 'X' in desc:
            axis_x.scale = log_scale_x
            ticks = generate_log_ticks(log_scale_x)
            axis_x.tick_values = ticks
        
        elif 'Y' in desc:
            axis_y.scale = log_scale_y
            ticks = generate_log_ticks(log_scale_y)
            axis_y.tick_values = ticks

    
    for mark in main_plot.marks:
        mark.scales = {
            'x': axis_x.scale,
            'y': axis_y.scale
        }
    
    main_plot.scale_x = axis_x.scale
    main_plot.scale_y = axis_y.scale
    
   #right_block.children = [
   #    plot_name,
   #    widgets.HBox(
   #        [
   #            axis_x_name,
   #            x_scale_dropdown
   #        ],
   #    ),
   #    widgets.HBox(
   #        [
   #            axis_y_name,
   #            y_scale_dropdown
   #        ],
   #    ),
   #    main_plot,
   #    mark_dropdown,
   #    regression_box,
   #    style_box
   #]

In [53]:
def calculate_limits(x_values, y_values): #, log_x=False, log_y=False):
    """this function calculates the limits of the axes from
    the given values of x and y: a list containing the arrays
    of x and y.
    """

    max_x = np.amax(x_values)
    min_x = np.min(x_values)

    max_y = np.amax(y_values)
    min_y = np.min(y_values)

    start_x = min_x #- 0.1*abs(max_x - min_x)
    end_x = max_x #+ 0.1*abs(max_x - min_x)

    start_y = min_y #- 0.1*abs(max_y - min_y)
    end_y = max_y #+ 0.1*abs(max_y - min_y)
    
    #if log_x:
    #    
    #    start_x = min_x/1.2
    #    end_x = max_x*1.2
    #    
    #if log_y:
    #    
    #    start_y = min_y/1.2
    #    end_y = max_y*1.2
        
    return start_x, end_x, start_y, end_y

In [54]:
lin_scale_x = bqs.LinearScale(
    min = 0,
    max = 10,
    stabilized = True
)

lin_scale_y = bqs.LinearScale(
    min = 0,
    max = 10,
    stabilized = True
)

log_scale_x = bqs.LogScale(min = 1, max = 100)
log_scale_y = bqs.LogScale(min = 1, max = 100)

axis_x = bqa.Axis(
    scale=lin_scale_x,
    tick_format='0.2f',
    tick_style={'font-size': '15px'},
    num_ticks = 5,
    grid_lines = 'none',
    grid_color = '#8e8e8e', 
    label='x',
    label_location='middle',
    label_style={'stroke': 'black', 'default-size': 35},
    label_offset='50px'
)

axis_y = bqa.Axis(
    scale=lin_scale_y,
    tick_format='0.2f',
    tick_style={'font-size': '15px'},
    num_ticks = 4,
    grid_lines = 'none',
    grid_color = '#8e8e8e', 
    orientation='vertical',
    label='y',
    label_location='middle',
    label_style={'stroke': 'red', 'default_size': 35},
    label_offset='50px'
)

main_plot = bq.Figure(
    title='',
    marks=[],
    axes=[axis_x, axis_y],
    animation_duration=500,
    layout = widgets.Layout(width='80%', align_self='center'),
    legend_location='top-right',
    background_style= {'fill': 'white',  'stroke': 'black'},
    fig_margin=dict(top=80, bottom=80, left=75, right=20)
)

In [55]:
def prepare_export(a):
    
    for mark in main_plot.marks:
        if mark.selected is not None:
            if len(mark.selected) == 0:
                mark.selected = None
                
    main_plot.interaction = None
    
    global data
    
    text_lines = []
    if len(regression_expression.value) > 7:
        text_lines.append(regression_expression.value)
        for error in errors_expression.children:
            text_lines.append(error.value)
    
    data = repr((main_plot, text_lines))
    %store data

    cwd = os.getcwd()
    export_button.value = "<form action=" + "../apps/modules/export_module.ipynb" + " target='_blank'><button type=''submit''>Open in export module</button></form>"

In [56]:
right_block = widgets.VBox(
    [],
    layout = widgets.Layout(
        border='solid 1px',
        padding='20px',
        align_items = 'center',
        width='50%'
    )    
)

plot_name = widgets.Text(
    value='My plot',
    description="<b>Plot's name:</b>",
    disabled=False
)

widgets.jslink((plot_name, 'value'), (main_plot, 'title'))

axis_x_name = widgets.Text(
    value='x',
    description="X axis:",
    disabled=False
)

widgets.jslink((axis_x_name, 'value'), (axis_x, 'label'))

x_scale_dropdown = widgets.Dropdown(
    value = 'Linear scale',
    options = ['Linear scale', 'Logarithmic scale'],
    description='<h5>X scale: </h5>',
    rows = 10,
    layout = widgets.Layout(width = '200px')
)

x_scale_dropdown.observe(change_scale, 'value')

axis_y_name = widgets.Text(
    value='y',
    description="Y axis:",
    disabled=False
)

widgets.jslink((axis_y_name, 'value'), (axis_y, 'label'))

y_scale_dropdown = widgets.Dropdown(
    value = 'Linear scale',
    options = ['Linear scale', 'Logarithmic scale'],
    description='<h5>Y scale: </h5>',
    rows = 10,
    layout = widgets.Layout(width = '200px')
)

y_scale_dropdown.observe(change_scale, 'value')

prepare_export_button = widgets.Button(
        description='Prepare to export',
        disabled=False,
        button_style='',
        tooltip='',
    )

prepare_export_button.on_click(prepare_export)

export_button = widgets.HTML(
    value = ""
)

In [57]:
def update_regression(change):
    
    if show_regression_checkbox.value:
    
        i = mark_dropdown.index

        x_values = main_plot.marks[2*i].x
        y_values = main_plot.marks[2*i].y

        order = regression_order_slider.value

        reg_x, reg_y, poly, errors = calculate_regression(
            x_values,
            y_values,
            order
        )

        # Update regression mark
        main_plot.marks[2*i+1].x = reg_x
        main_plot.marks[2*i+1].y = reg_y

        # Update regression expression and errors
        reg_exp = '\(y = \;\)'
        chars = 'abcdef'

        errors_expression.children = []

        power = len(poly)

        for i in range(len(poly)):
            
            decimal = regression_decimals_dropdown.value
            
            coef = np.around([poly[i],],decimal)[0] 
            error = np.around([errors[i],], decimal)[0]

            power = power - 1

            if power == 1:

                reg_exp = reg_exp + '\(' + chars[i] + '\; x \; + \; \)' 

            elif power == 0:
                reg_exp = reg_exp + '\(' + chars[i] + ' \)'

            else:
                reg_exp = reg_exp + '\(' + chars[i] + '\; x^{' + str(power) + '} \; + \; \)'

            err_exp = '\(' + chars[i] + '\; = \; ' + str(coef) + '\; \pm \;' + str(error) + '\)'
            errors_expression.children = errors_expression.children + (widgets.HTMLMath(value = err_exp),)

        regression_expression.value = reg_exp
        
    else:
        
        i = mark_dropdown.index
        
        main_plot.marks[2*i+1].x = []
        main_plot.marks[2*i+1].y = []
        
        errors_expression.children = []
        regression_expression.value = '\(y =\)'

In [58]:
def calculate_regression(x_values, y_values, order):
    
    try:
        if len(x_values) > order + 1:
            poly, cov = np.polyfit(x_values, y_values, order, cov=True)

        else:
            poly = np.polyfit(x_values, y_values, len(x_values) - 1)
            cov = np.zeros([len(poly),len(poly)])

    except:
        return None
    
    reg_x = np.linspace(
        start = np.min(x_values),
        stop = np.max(x_values),
        num = 1000
    )
    
    reg_y = np.polyval(poly, reg_x)
    
    errors = np.sqrt(np.diag(cov))
    
    return reg_x, reg_y, poly, errors

In [59]:
def update_mark_dropdown():

    mark_dropdown.options = [mark.labels[0] for mark in main_plot.marks[::2]]
    #mark_dropdown.value = mark_dropdown.options[0]


In [60]:
def update_mark_settings(change):
    
    ## first we save the old settings
    if change is not None:
        if change.old is not None:

            if 'Scatter' in change.old and change.old in mark_dropdown.options:
                plot_name = choose_plot_dropdown.value
                p = plot_database.get_plot(plot_name)
                marks = p.marks

                i = mark_dropdown.options.index(change.old)
                mark = marks[i]

                mark.reg = show_regression_checkbox.value
                mark.order = regression_order_slider.value
                mark.dec = regression_decimals_dropdown.value
                
                plot_database.plots[plot_name].marks[i] = mark
                plot_database.all[plot_name].marks[i] = mark
    
    # then we update the new ones
    plot_name = choose_plot_dropdown.value
    p = plot_database.get_plot(plot_name)
    marks = p.marks

    i = mark_dropdown.index
    
    if i is not None:
        mark = marks[i]
        
        show_regression_checkbox.value, regression_order_slider.value = mark.reg, mark.order

        update_regression(None)
        
        scat = main_plot.marks[2*i]
        reg = main_plot.marks[2*i+1]
        
        scatter_color.value = scat.default_colors[0]
        scatter_opacity.value = scat.default_opacities[0]
        scatter_width.value = scat.stroke_width
        
        regression_color.value = reg.colors[0]
        regression_opacity.value = reg.opacities[0]
        regression_width.value = reg.stroke_width
        
        regression_decimals_dropdown.value = mark.dec

In [61]:
def save_mark_settings(change):

    
    plot_name = choose_plot_dropdown.value
    
    if change.old in choose_plot_dropdown.options:

        plot_name = change.old
    
    p = plot_database.get_plot(plot_name)
    marks = p.marks
    
    i = mark_dropdown.index
    
    if i is None:
        i = 0
        
    mark = marks[i]
    
    mark.reg = show_regression_checkbox.value
    mark.order = regression_order_slider.value
    mark.dec = regression_decimals_dropdown.value
    
    plot_database.plots[plot_name].marks[i] = mark
    plot_database.all[plot_name].marks[i] = mark


In [62]:
mark_dropdown = widgets.Dropdown(
    options = [mark.labels[0] for mark in main_plot.marks[::2]],
    rows = 10,
    layout = widgets.Layout(width = '200px')
)

mark_dropdown.observe(update_mark_settings, 'value')
#mark_dropdown.observe(save_mark_settings, 'options')

In [63]:
regression_box = widgets.VBox(
    [],
    layout = widgets.Layout(
        align_items = 'center'
    )    
)

show_regression_checkbox = widgets.Checkbox(
    value=False,
    description='Show regression',
    layout = widgets.Layout(width = '200px')
)

show_regression_checkbox.observe(update_regression, 'value')
show_regression_checkbox.observe(save_mark_settings, 'value')

regression_order_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout = widgets.Layout(width = '200px')
)

regression_order_slider.observe(update_regression, 'value')
regression_order_slider.observe(save_mark_settings, 'value')

regression_decimals_dropdown = widgets.Dropdown(
    options = [i for i in range(0,14)],
    value = 2,
    #description='Number of decimals:',
)

regression_decimals_dropdown.observe(update_regression, 'value')

regression_expression = widgets.HTMLMath(
    value = '\(y =\)',
    description = 'Expression:'
)
        
errors_expression = widgets.VBox([])

regression_box.children = [
    show_regression_checkbox,
    widgets.HBox([
        widgets.Label('Order of regression'),
        regression_order_slider,
    ]),
    widgets.HBox([
        widgets.Label('Number of decimals:'),
        regression_decimals_dropdown,
    ]),
    regression_expression,
    errors_expression
]

In [64]:
def change_scatter_opacity(change):
    
    obj = change.owner
    i = mark_dropdown.index

    main_plot.marks[2*i].default_opacities = [obj.value]

In [65]:
def change_scatter_color(change):
    
    obj = change.owner
    i = mark_dropdown.index
    
    main_plot.marks[2*i].default_colors = [obj.value]

In [66]:
def change_scatter_width(change):
    
    obj = change.owner
    i = mark_dropdown.index
    
    main_plot.marks[2*i].stroke_width = obj.value

In [67]:
def change_regression_opacity(change):
    
    obj = change.owner
    i = mark_dropdown.index

    main_plot.marks[2*i+1].opacities = [obj.value]

In [68]:
def change_regression_color(change):
    
    obj = change.owner
    i = mark_dropdown.index
    
    main_plot.marks[2*i+1].colors = [obj.value]

In [69]:
def change_regression_width(change):
    
    obj = change.owner
    i = mark_dropdown.index
    
    main_plot.marks[2*i+1].stroke_width = obj.value

In [70]:
style_box = widgets.VBox(
    [],
    layout = widgets.Layout(
        align_items = 'center'
    )    
)

data_text = widgets.HTML(
    value = '<b>Data: </b>'
)

scatter_color = widgets.ColorPicker(
    concise=False,
    description='',
    value='#fad87c',
    disabled=False,
    layout = widgets.Layout(width = '100px')
)

scatter_color.observe(change_scatter_color, 'value')

scatter_opacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width = '350px')
)

scatter_opacity.observe(change_scatter_opacity, 'value')

scatter_width = widgets.FloatSlider(
    value=1.5,
    min=0,
    max=3.0,
    step=0.01,
    description='Width:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width = '350px')
)

scatter_width.observe(change_scatter_width, 'value')

regression_text = widgets.HTML(
    value = '<b>Regression: </b>'
)

regression_color = widgets.ColorPicker(
    concise=False,
    description='',
    value='#fad87c',
    disabled=False,
    layout = widgets.Layout(width = '100px')
)

regression_color.observe(change_regression_color, 'value')


regression_opacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width = '350px')
)

regression_opacity.observe(change_regression_opacity, 'value')

regression_width = widgets.FloatSlider(
    value=1.5,
    min=0,
    max=3.0,
    step=0.01,
    description='Width:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width = '350px')
)

regression_width.observe(change_regression_width, 'value')

style_box.children = [
    data_text,
    widgets.HBox([
        scatter_color,
        scatter_opacity,
    ]),
    scatter_width,
    regression_text,
    widgets.HBox([
        regression_color,
        regression_opacity
    ]),
    regression_width
]

In [71]:
right_block.children = [
    plot_name,
    widgets.HBox(
        [
            axis_x_name,
            x_scale_dropdown
        ],
    ),
    widgets.HBox(
        [
            axis_y_name,
            y_scale_dropdown
        ],
    ),
    widgets.HBox(
        [
            prepare_export_button,
            export_button
        ],
    ),
    main_plot,
    mark_dropdown,
    regression_box,
    style_box
]

In [72]:
middle_block = widgets.HBox(
    [],
    layout = widgets.Layout(
        width='95%'
    )    
)

middle_block.children = [
    left_block,
    right_block
]

In [73]:
main_block = widgets.VBox(
    [],
    layout = widgets.Layout(
        width='100%'
    )    
)

main_block.children = [
    top_block,
]

In [74]:
main_block

Stored 'data' (str)
Stored 'data' (str)
